In [2]:
import pandas as pd

In [3]:
#link do folderu, w którym mamy bazę:

%cd "C:\Users\kdabr\Documents\DataScience2021\ProjektPython"

C:\Users\kdabr\Documents\DataScience2021\ProjektPython


In [4]:
df=pd.read_csv("./database.sqlite")

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.drop(columns=["Unnamed: 0","HOSPSTAY",'DOF1', 'GAD1','DVEST', 
                 'DVBASIS','BAGDEPLY','BAGFAIL','CASENO','MANAVAIL', 'MANFAIL', 'TREATMNT','DEATH'
                ],inplace = True)




Aby sprawnie pracować na danych zawartych w tabeli, podzielimy ją na 3 tabele połacząne kluczem głównym (CASEID i PSU)



In [6]:
occupants = df[["CASEID", 'PSU','AGE', 'BAGAVAIL','CHTYPE', 'HEIGHT', 'INJSEV',
       'MANUSE', 'OCCNO', 'ROLE', 'SEATPOS', 'SEX', 'WEIGHT']]

In [7]:
occupants.drop_duplicates(subset=["CASEID", 'PSU','AGE', 'BAGAVAIL','CHTYPE', 'HEIGHT', 'INJSEV',
       'MANUSE', 'OCCNO', 'ROLE', 'SEATPOS', 'SEX', 'WEIGHT'],inplace=True, ignore_index=True)

<ipython-input-7-6732b027358d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  occupants.drop_duplicates(subset=["CASEID", 'PSU','AGE', 'BAGAVAIL','CHTYPE', 'HEIGHT', 'INJSEV',


In [8]:
vehicles= df[['CASEID', 'PSU','BODYTYPE', 'MAKE' , 'DVTOTAL','MODELYR']]

In [9]:
vehicles.drop_duplicates(subset=['CASEID', 'PSU','BODYTYPE', 'MAKE' , 'DVTOTAL','MODELYR'],inplace=True, ignore_index=True)

<ipython-input-9-650ac2603853>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles.drop_duplicates(subset=['CASEID', 'PSU','BODYTYPE', 'MAKE' , 'DVTOTAL','MODELYR'],inplace=True, ignore_index=True)


In [10]:
accidents = df[['CASEID', 'PSU', 'STRATIF','YEAR']]

In [11]:
accidents.drop_duplicates(subset=['CASEID', 'PSU', 'STRATIF','YEAR'], inplace =True, ignore_index=True)

<ipython-input-11-0fba22f11641>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accidents.drop_duplicates(subset=['CASEID', 'PSU', 'STRATIF','YEAR'], inplace =True, ignore_index=True)


## Wspolne funkcje - grupowanie

In [1]:
def classify_value(input_value,mappings): 
    
    for mark,values in  mappings.items():
        if input_value in values: 
            return mark 
         
    return None

## Kolumna bodytype


In [12]:
bodytype_mappings = {0:[1,2,3,4,5,6,7,8,9,17],                                       # osobowy
                          1:[61,62,63,64,69,74,78,79],                               # ciezarowka 
                          2:[24,25,28,50,58,59,60],                                  # busy   
                          3:[10,40,41,45,48],                                        # dostawcze 
                          4:[65,68,67,70,80,81,82,89,90],                            # motory&quady
                          5:[12, 14, 15, 16, 19, 20, 21, 22, 29, 30, 31, 32, 33,39], # większe samochody osobowe
                          6:[92, 93, 97, 42, 23, 11]}                                # pozostale 
    

vehicles.head(5)
vehicles["BODYTYPE_NEW"] = vehicles["BODYTYPE"].apply(classify_value,mappings=bodytype_mappings)
vehicles[['BODYTYPE','BODYTYPE_NEW']].value_counts()

<ipython-input-12-256bccace185>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles["BODYTYPE_NEW"] = vehicles["BODYTYPE"].apply(classify_value,mappings=bodytype_mappings)


BODYTYPE  BODYTYPE_NEW
4.0       0.0             39903
14.0      5.0             13684
31.0      5.0              7341
2.0       0.0              6800
20.0      5.0              4689
30.0      5.0              4048
15.0      5.0              3489
6.0       0.0              2301
3.0       0.0              2147
5.0       0.0              2100
21.0      5.0              1419
68.0      4.0              1323
1.0       0.0              1020
9.0       0.0               652
16.0      5.0               551
63.0      1.0               352
78.0      1.0               300
58.0      2.0               236
61.0      1.0               221
64.0      1.0               190
39.0      5.0               184
80.0      4.0               138
62.0      1.0               136
50.0      2.0               134
40.0      3.0               124
28.0      2.0               102
67.0      4.0                92
19.0      5.0                90
74.0      1.0                58
22.0      5.0                56
69.0      1.0    

### Kontrola pustych wartosci

In [13]:
vehicles['BODYTYPE_NEW'].isna().value_counts()


False    94350
True       193
Name: BODYTYPE_NEW, dtype: int64

In [16]:
vehicles['BODYTYPE'].isna().value_counts()

False    94350
True       193
Name: BODYTYPE, dtype: int64

## Kolumna STRATIF 

In [17]:
stratif_mappings =       {0:['A','B'], # smiertelne obrażenia
                          1:['J','K'], # poważne obrażenia – hospitalizowany (min. 1noc)
                          2:['C','D'], #  poważne obrażenia – nie hospitalizowany
                          3:['E','F'], # niewielkie obrażenia, ale przewieziony do szpitala
                          4:['G','H']} # niewielkie obrażenia

accidents.head(5)
accidents["STRATIF_NEW"] = accidents["STRATIF"].apply(classify_value,mappings=stratif_mappings)
accidents[['STRATIF','STRATIF_NEW']].value_counts()

<ipython-input-17-86d1d05ea930>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accidents["STRATIF_NEW"] = accidents["STRATIF"].apply(classify_value,mappings=stratif_mappings)


STRATIF  STRATIF_NEW
F        3.0            10079
H        4.0             9746
K        1.0             7434
G        4.0             6161
E        3.0             6066
D        2.0             4174
B        0.0             3156
C        2.0             2690
J        1.0             1785
A        0.0              685
dtype: int64

### Kontrola pustych wartosci

In [18]:
accidents['STRATIF_NEW'].isna().value_counts()

False    51976
True         2
Name: STRATIF_NEW, dtype: int64

In [19]:
accidents['STRATIF'].isna().value_counts()

False    51976
True         2
Name: STRATIF, dtype: int64